# sx.nciyes.process.ipynb

Notebook for processing input datas.

In [28]:

nci_fpath = "../../data/ExptNCI/merged_0304.csv"
nci_save_fpath = "./Inputs/Dev/dev.nci.csv"

pdbbind_path = "../../data/pdbbind2020/refined-set/" # for .pdb files
#hbligand_path = "../../data/HBLigand/" # for .

pdbs_save_path = "./Inputs/Dev/RenamedPDBBind/"
pdb_list_save_fpath = "./Inputs/Dev/dev.pdbs.csv"

ligands_save_path = "./Inputs/Dev/RenamedPDBBind/"
ligand_list_save_fpath = "./Inputs/Dev/dev.ligands.csv"

target_pdb_list = ['1g35']

### NCI data processing

In [30]:
import pandas as pd
nci_data = pd.read_csv(nci_fpath)
nci_data = nci_data[nci_data.PDB_Code.isin(target_pdb_list)]
nci_data = nci_data[(nci_data["ED_2.5A"]>=0)|(nci_data["ED_3.5A"]>=0)]
nci_data = nci_data[nci_data.Group=="protein"]
nci_data["ResFullID"] = nci_data["ChainID"]+"_"+nci_data["ResID"].astype(str)
nci_data.to_csv(nci_save_fpath)
print(f"Processed NCIs file save to {nci_save_fpath}")

Processed NCIs file save to ./Inputs/Dev/dev.nci.csv


### PDB and ligand file processing

In [26]:
import re
import os
import shutil

class MolStructureError(Exception):
    def __init__(self, errorinfo):
        super().__init__()
        self.errorinfo = errorinfo
    def __str__(self):
        return self.errorinfo

In [27]:
pdb_df_list = []
ligand_df_list = []
print("Processing .pdb and .mol2 files:")
for _pdb in target_pdb_list:
    print(f"  Processing pdb {_pdb}:")
    shutil.copy(f"{pdbbind_path}{_pdb}/{_pdb}_protein.pdb", f"{pdbs_save_path}{_pdb}_protein.pdb")
    print(f"    Copy {pdbbind_path}{_pdb}/{_pdb}_protein.pdb to {pdbs_save_path}{_pdb}_protein.pdb")
    with open(f"{pdbbind_path}{_pdb}/{_pdb}_ligand.mol2", "r") as f:
        for i in range(13):
            text = f.readline()
        text = text.split()
        if (text[0] == "1") and (text[-2] != "UNK"):
            ligand_name = text[-2]
    shutil.copy(f"{pdbbind_path}{_pdb}/{_pdb}_ligand.mol2", f"{ligands_save_path}{_pdb}_ligand_{ligand_name}.mol2")
    print(f"    Copy {pdbbind_path}{_pdb}/{_pdb}_ligand.mol2 to {ligands_save_path}{_pdb}_ligand_{ligand_name}.mol2")
    pdb_df_list.append([_pdb, f"{pdbs_save_path}{_pdb}_protein.pdb"])
    ligand_df_list.append([_pdb, ligand_name, f"{ligands_save_path}{_pdb}_ligand_{ligand_name}.mol2"])
    
pdb_df = pd.DataFrame(pdb_df_list, columns=["pdb_code", "pdb_fpath"])
pdb_df.to_csv(pdb_list_save_fpath)
print(f"Protein list infomation saved to {pdb_list_save_fpath}.")
ligand_df = pd.DataFrame(ligand_df_list, columns=["pdb_code", "ligand_name", "ligand_fpath"])
ligand_df.to_csv(ligand_list_save_fpath)
print(f"Ligand list infomation saved to {ligand_list_save_fpath}.")

Processing .pdb and .mol2 files:
  Processing pdb 1g35:
    Copy ../../data/pdbbind2020/refined-set/1g35/1g35_protein.pdb to ./Inputs/Dev/RenamedPDBBind/1g35_protein.pdb
    Copy ../../data/pdbbind2020/refined-set/1g35/1g35_ligand.mol2 to ./Inputs/Dev/RenamedPDBBind/1g35_ligand_AHF.mol2
Protein list infomation saved to ./Inputs/Dev/dev.pdbs.csv.
Ligand list infomation saved to ./Inputs/Dev/dev.ligands.csv.
